In [ ]:
import torch
import stable_baselines3
import sys
import numpy as np

print("python version:", sys.version)
print("stable_baselines3 version:", stable_baselines3.__version__)
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version:", torch.version.cuda)
print("cudnn version:", torch.backends.cudnn.version())

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# set torch default device
torch.set_default_device(device)

# Equation of Motion 3D Quadcopter

In [ ]:
# reload sympy
from sympy import *

# Equations of motion 3D quadcopter + INDI inner loop

state = symbols('x y z v_x v_y v_z phi theta psi p q r T_norm')
x,y,z,vx,vy,vz,phi,theta,psi,p,q,r,T_norm = state
# commands normalized to [-1,1]
control = symbols('p_cmd,q_cmd,r_cmd,T_cmd')
p_cmd,q_cmd,r_cmd,T_cmd = control

# Physics
g = 9.81

# INDI params
taup = 0.03
tauq = 0.03
taur = 0.03
tauT = 0.03

p_min =-3.0 #-6.0
p_max = 3.0 #6.0
q_min =-3.0 #-6.0
q_max = 3.0 #6.0
r_min =-2.0
r_max = 2.0
T_min = 0.0
T_max = 16.0

# Drag model
k_x = 0.33915248
k_y = 0.4314916

# Rotation matrix 
Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
Ry = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0], [sin(psi), cos(psi), 0], [0, 0, 1]])
R = Rz*Ry*Rx

# Body velocity
vbx, vby, vbz = R.T@Matrix([vx,vy,vz])

# Drag
Dx = -k_x*vbx
Dy = -k_y*vby

# Dynamics
d_x = vx
d_y = vy
d_z = vz

T = (T_norm+1)/2*(T_max-T_min)+T_min
d_vx, d_vy, d_vz = Matrix([0,0,g]) + R@Matrix([Dx, Dy,-T])

d_phi   = p + q*sin(phi)*tan(theta) + r*cos(phi)*tan(theta)
d_theta = q*cos(phi) - r*sin(phi)
d_psi   = q*sin(phi)/cos(theta) + r*cos(phi)/cos(theta)

d_p      = ((p_cmd+1)/2*(p_max-p_min)+p_min - p)/taup
d_q      = ((q_cmd+1)/2*(q_max-q_min)+q_min - q)/tauq
d_r      = ((r_cmd+1)/2*(r_max-r_min)+r_min - r)/taur
d_T_norm = (T_cmd - T_norm)/tauT

# State space model
f = [d_x, d_y, d_z, d_vx, d_vy, d_vz, d_phi, d_theta, d_psi, d_p, d_q, d_r, d_T_norm]

# lambdify
f_func = lambdify((Array(state), Array(control)), Array(f), 'numpy')

In [ ]:
axs, ays, azs = R*Matrix([0,0,x])
print(ccode(axs))
print(ccode(ays))
print(ccode(azs))


# Define the 3D Quad Gates Environment

In [ ]:
# Efficient vectorized version of the environment
from gym import spaces
from stable_baselines3.common.vec_env import VecEnv

class Quadcopter3DGates(VecEnv):
    def __init__(self,
                 num_envs,
                 gates_pos,
                 gate_yaw,
                 start_pos,
                 gates_ahead=0,
                 pause_if_collision=False,
                 ):
        
        # Define the race track
        self.start_pos = start_pos.astype(np.float32)
        self.gate_pos = gates_pos.astype(np.float32)
        self.gate_yaw = gate_yaw.astype(np.float32)
        self.num_gates = gates_pos.shape[0]
        self.gates_ahead = gates_ahead
        
        # Pause if collision
        self.pause_if_collision = pause_if_collision

        # Calculate relative gates
        # pos,yaw of gate i in reference frame of gate i-1 (assumes a looped track)
        self.gate_pos_rel = np.zeros((self.num_gates,3), dtype=np.float32)
        self.gate_yaw_rel = np.zeros(self.num_gates, dtype=np.float32)
        for i in range(0,self.num_gates):
            self.gate_pos_rel[i] = self.gate_pos[i] - self.gate_pos[i-1]
            # Rotation matrix
            R = np.array([
                [np.cos(self.gate_yaw[i-1]), np.sin(self.gate_yaw[i-1])],
                [-np.sin(self.gate_yaw[i-1]), np.cos(self.gate_yaw[i-1])]
            ])
            self.gate_pos_rel[i,0:2] = R@self.gate_pos_rel[i,0:2]
            self.gate_yaw_rel[i] = self.gate_yaw[i] - self.gate_yaw[i-1]

        # Define the target gate for each environment
        self.target_gates = np.zeros(num_envs, dtype=int)

        # action space: [p_cmd,q_cmd,r_cmd,T_cmd] normalized to [-1,1]
        action_space = spaces.Box(low=-1, high=1, shape=(4,))

        # observation space: pos[G], vel[G], att[eulerB->G], rates[B], rpms, future_gates[G], future_gate_dirs[G]
        # [G] = reference frame aligned with target gate
        # [B] = body frame
        self.state_len = 13+4*self.gates_ahead
        observation_space = spaces.Box(
            low  = np.array([-np.inf]*self.state_len),
            high = np.array([ np.inf]*self.state_len)
        )

        # Initialize the VecEnv
        VecEnv.__init__(self,num_envs, observation_space, action_space)

        # world state: pos[W], vel[W], att[eulerB->W], rates[B], rpms
        self.world_states = np.zeros((num_envs,13), dtype=np.float32)
        # observation state
        self.states = np.zeros((num_envs,self.state_len), dtype=np.float32)

        # Define any other environment-specific parameters
        self.max_steps = 1200      # Maximum number of steps in an episode
        self.dt = np.float32(0.01) # Time step duration

        self.step_counts = np.zeros(num_envs, dtype=int)
        self.actions = np.zeros((num_envs,4), dtype=np.float32)
        self.dones = np.zeros(num_envs, dtype=bool)
        self.final_gate_passed = np.zeros(num_envs, dtype=bool)

        self.update_states = self.update_states_gate

        self.pause = False

    def update_states_world(self):
        self.states = self.world_states

    def update_states_gate(self):
        # Transform pos and vel in gate frame
        gate_pos = self.gate_pos[self.target_gates%self.num_gates]
        gate_yaw = self.gate_yaw[self.target_gates%self.num_gates]

        # Rotation matrix from world frame to gate frame
        R = np.array([
            [np.cos(gate_yaw), np.sin(gate_yaw)],
            [-np.sin(gate_yaw), np.cos(gate_yaw)]
        ]).transpose((2,1,0))

        # new state array to prevent the weird bug related to indexing ([:] syntax)
        new_states = np.zeros_like(self.states)

        # Update positions
        pos_W = self.world_states[:,0:3]
        pos_G = (pos_W[:,np.newaxis,0:2] - gate_pos[:,np.newaxis,0:2]) @ R
        new_states[:,0:2] = pos_G[:,0,:]
        new_states[:,2] = pos_W[:,2] - gate_pos[:,2]

        # Update velocities
        vel_W = self.world_states[:,3:6]
        vel_G = (vel_W[:,np.newaxis,0:2]) @ R
        new_states[:,3:5] = vel_G[:,0,:]
        new_states[:,5] = vel_W[:,2]

        # Update attitude
        new_states[:,6:8] = self.world_states[:,6:8]
        yaw = self.world_states[:,8] - gate_yaw
        yaw %= 2*np.pi
        yaw[yaw > np.pi] -= 2*np.pi
        yaw[yaw < -np.pi] += 2*np.pi
        new_states[:,8] = yaw

        # Update rates
        new_states[:,9:12] = self.world_states[:,9:12]

        # Update Thrust
        new_states[:,12] = self.world_states[:,12]

        # Update future gates relative to current gate ([0,0,0,0] for out of bounds)
        for i in range(self.gates_ahead):
            indices = (self.target_gates+i+1)
            # loop when out of bounds
            indices = indices % self.num_gates
            valid = indices < self.num_gates
            new_states[valid,13+4*i:13+4*i+3] = self.gate_pos_rel[indices[valid]]
            new_states[valid,13+4*i+3] = self.gate_yaw_rel[indices[valid]]

        self.states = new_states

    def reset_(self, dones):
        num_reset = dones.sum()

        x0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[0]
        y0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[1]
        z0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[2]
        
        vx0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vy0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vz0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        
        phi0   = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,))
        theta0 = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,))
        psi0   = np.random.uniform(-np.pi,np.pi, size=(num_reset,))
        
        p0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        q0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        r0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        
        T0 = np.random.uniform(-.1,.1, size=(num_reset,))

        self.world_states[dones] = np.stack([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, T0], axis=1)

        self.step_counts[dones] = np.zeros(num_reset)
        
        self.target_gates[dones] = np.zeros(num_reset, dtype=int)

        # update states
        self.update_states()
        return self.states
    
    def reset(self):
        return self.reset_(np.ones(self.num_envs, dtype=bool))

    def step_async(self, actions):
        self.actions = actions
    
    def step_wait(self):
        new_states = self.world_states + self.dt*f_func(self.world_states.T, self.actions.T).T
        self.step_counts += 1

        pos_old = self.world_states[:,0:3]
        pos_new = new_states[:,0:3]
        pos_gate = self.gate_pos[self.target_gates%self.num_gates]
        yaw_gate = self.gate_yaw[self.target_gates%self.num_gates]

        # Rewards from [...]
        d2g_old = np.linalg.norm(pos_old - pos_gate, axis=1)
        d2g_new = np.linalg.norm(pos_new - pos_gate, axis=1)
        rat_penalty = 0*0.01*np.linalg.norm(new_states[:,9:12], axis=1)
        rewards = d2g_old - d2g_new - rat_penalty
        
        normal = np.array([np.cos(yaw_gate), np.sin(yaw_gate)]).T
        # dot product of normal and position vector over axis 1
        pos_old_projected = (pos_old[:,0]-pos_gate[:,0])*normal[:,0] + (pos_old[:,1]-pos_gate[:,1])*normal[:,1]
        pos_new_projected = (pos_new[:,0]-pos_gate[:,0])*normal[:,0] + (pos_new[:,1]-pos_gate[:,1])*normal[:,1]
        passed_gate_plane = (pos_old_projected < 0) & (pos_new_projected > 0)
        gate_passed = passed_gate_plane & np.all(np.abs(pos_new - pos_gate)<0.5, axis=1)
        gate_collision = passed_gate_plane & np.any(np.abs(pos_new - pos_gate)>0.5, axis=1)
        
        # gate reward (+ dist penalty)
        rewards[gate_passed] = 10 - 10*d2g_new[gate_passed]
        
        # Check for gate collision
        rewards[gate_collision] = -10

        # Check ground collision (z > 0)
        ground_collision = new_states[:,2] > 0
        rewards[ground_collision] = -10
        
        # Check out of bounds
        # outside grid abs(x,y)>10
        # prevent numerical issues: abs(p,q,r) < 1000
        out_of_bounds = np.any(np.abs(new_states[:,0:2]) > 10, axis=1) | np.any(np.abs(new_states[:,9:12]) > 1000, axis=1)
        rewards[out_of_bounds] = -10
        
        # Check number of steps
        max_steps_reached = self.step_counts >= self.max_steps

        # Update target gate
        self.target_gates[gate_passed] += 1
        self.target_gates[gate_passed] %= self.num_gates
        
        # Check if final gate has been passed
        # self.final_gate_passed = self.target_gates >= self.num_gates

        # give reward for passing final gate
        rewards[self.final_gate_passed] = 10
        
        # Check if the episode is done
        dones = max_steps_reached | ground_collision | gate_collision | out_of_bounds #| self.final_gate_passed
        self.dones = dones
        
        # Pause if collision
        if self.pause:
            dones = dones & ~dones
            self.dones = dones
        elif self.pause_if_collision:
            # dones = max_steps_reached | final_gate_passed | out_of_bounds
            update = ~dones #~(gate_collision | ground_collision)
            # Update world states
            self.world_states[update] = new_states[update]
            self.update_states()
            # Reset env if done (and update states)
            # self.reset_(dones)
        else:
            # Update world states
            self.world_states = new_states
            # reset env if done (and update states)
            self.reset_(dones)


        # Write info dicts
        infos = [{}] * self.num_envs
        for i in range(self.num_envs):
            if dones[i]:
                infos[i]["terminal_observation"] = self.states[i]
            if max_steps_reached[i]:
                infos[i]["TimeLimit.truncated"] = True
        return self.states, rewards, dones, infos
    
    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def get_attr(self, attr_name, indices=None):
        pass

    def set_attr(self, attr_name, value, indices=None):
        pass

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        pass

    def env_is_wrapped(self, wrapper_class, indices=None):
        return [False]*self.num_envs

    def render(self, mode='human'):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','T'], self.world_states.T))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.actions.T)+1)/2))
        return {**state_dict, **action_dict}

# Define Race Track

In [ ]:
import importlib
from quadcopter_animation import animation
importlib.reload(animation)

# Define the race track
# gate_pos = np.array([
#     [-1.5,-2,-1.5],
#     [1.5,2,-1.5],
#     [1.5,-2,-2.5],
#     [-1.5,2,-1.5]
# ]*2)

gate_pos = np.array([
    [ 2,-1.5,-1.5],
    [ 2, 1.5,-1.5],
    [-2, 1.5,-1.5],
    [-2,-1.5,-1.5]
]*2)

# gate_yaw = np.array([
#     0,
#     0,
#     np.pi,
#     np.pi
# ]*2)
gate_yaw = np.array([
    np.pi/4,
    3*np.pi/4,
    5*np.pi/4,
    7*np.pi/4
]*2)


# start_pos = gate_pos[0] - np.array([2,0,0])
start_pos = gate_pos[3] #- np.array([2,0,0])

num = 10
env = Quadcopter3DGates(num_envs=num, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, pause_if_collision=False)

# Run a random agent
env.reset()

done = False
def run():
    global done
    action = np.random.uniform(-1,1, size=(num,4))
    # action[:,3] = 1
    state, reward, done, _ = env.step(action)
    # print(state[0][-4:])
    # print(env.disturbances[0])
    if reward[0] > 1:
        print("reward:", reward)
    return env.render()

animation.view(run, gate_pos=gate_pos, gate_yaw=gate_yaw) #, record_steps=1000, show_window=True)

# Train PPO Model

In [ ]:
import os
from stable_baselines3 import PPO
from datetime import datetime
from stable_baselines3.common.vec_env import VecMonitor
import importlib
from quadcopter_animation import animation

models_dir = 'models/PPO_INDI'
log_dir = 'logs/PPO_INDI'
video_log_dir = 'videos/PPO_INDI'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(video_log_dir):
    os.makedirs(video_log_dir)

# Date and time string for unique folder names
datetime_str = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the environment
env = Quadcopter3DGates(num_envs=100, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1)
test_env = Quadcopter3DGates(num_envs=10, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1, pause_if_collision=True)

# Wrap the environment in a Monitor wrapper
env = VecMonitor(env)

# MODEL DEFINITION
policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[dict(pi=[120,120,120], vf=[120,120,120])], log_std_init = 0)
model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    verbose=0,
    tensorboard_log=log_dir,
    n_steps=1000,
    batch_size=5000,
    n_epochs=10,
    gamma=0.999
)

print(model.policy)
print(model.num_timesteps)

def animate_policy(model, env, deterministic=False, log_times=False, **kwargs):
    env.reset()
    def run():
        actions, _ = model.predict(env.states, deterministic=deterministic)
        states, rewards, dones, infos = env.step(actions)
        if log_times:
            if rewards[0] == 10:
                print(env.step_counts[0]*env.dt)
        return env.render()
    animation.view(run, gate_pos=env.gate_pos, gate_yaw=env.gate_yaw, **kwargs)

# animate untrained policy (use this to set the recording camera position)
animate_policy(model, test_env)

def train(model, test_env, log_name, n=10000000000):
    # save every 10 policy rollouts
    TIMESTEPS = model.n_steps*env.num_envs*10
    for i in range(0,n):
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=log_name)
        time_steps = model.num_timesteps
        # save model
        model.save(models_dir + '/' + log_name + '/' + str(time_steps))
        # save policy animation
        animate_policy(
            model,
            test_env,
            record_steps=1000,
            record_file=video_log_dir + '/' + log_name + '/' + str(time_steps) + '.mp4',
            show_window=False
        )

In [ ]:
# run training loop
# train(model, test_env, 'INDI_gate_penalty_no_rate_penalty')

# Simulate PPO model

In [ ]:
test_env = Quadcopter3DGates(num_envs=1, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1, pause_if_collision=True)

# path = 'models/PPO_INDI/INDI_TEST2_no_rate_penalty/375000000'
path = 'models/PPO_INDI/INDI_TEST2_with_rate_penalty/375000000'
path = 'models/PPO_INDI/INDI_TEST3_pq_max=3_rate_penalty/65000000'
path = 'models/PPO_INDI/INDI_TEST3_pq_max=3_no_rate_penalty/117000000'
path = 'models/PPO_INDI/INDI_gate_penalty_no_rate_penalty/305000000'


model = PPO.load(path)
animate_policy(model, test_env, deterministic=False, log_times=True)

In [ ]:
test_env.reset()
action_list = []

def run():
    global action_list
    actions, _ = model.predict(test_env.states, deterministic=False)
    action_list.append(actions[0])
    states, rewards, dones, infos = test_env.step(actions)
    return test_env.render()

animation.view(run, gate_pos=test_env.gate_pos, gate_yaw=test_env.gate_yaw)
action_list = np.array(action_list)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(action_list[:,0])
plt.plot(action_list[:,1])
plt.plot(action_list[:,2])

In [ ]:
# LOGGING
test_env.max_steps = 10000
test_env.reset()

time = []
state_traj = []
action_traj = []

def run():
    actions, _ = model.predict(test_env.states, deterministic=False)
    states, rewards, dones, infos = test_env.step(actions)
    state_traj.append(test_env.world_states[0].copy())
    action_traj.append(actions[0].copy())
    time.append(test_env.step_counts[0]*test_env.dt)
        
    return test_env.render()

animation.view(run, gate_pos=env.gate_pos, gate_yaw=env.gate_yaw)

# make log dict
log_dict = {}
state_traj = np.array(state_traj)
action_traj = np.array(action_traj)

log_dict['t'] = time
log_dict['x'] = state_traj[:,0]
log_dict['y'] = state_traj[:,1]
log_dict['z'] = state_traj[:,2]
log_dict['vx'] = state_traj[:,3]
log_dict['vy'] = state_traj[:,4]
log_dict['vz'] = state_traj[:,5]
log_dict['V'] = np.sqrt(state_traj[:,3]**2 + state_traj[:,4]**2 + state_traj[:,5]**2)
log_dict['phi'] = state_traj[:,6]
log_dict['theta'] = state_traj[:,7]
log_dict['psi'] = state_traj[:,8]
log_dict['u1'] = (action_traj[:,0]+1)/2
log_dict['u2'] = (action_traj[:,1]+1)/2
log_dict['u3'] = (action_traj[:,2]+1)/2
log_dict['u4'] = (action_traj[:,3]+1)/2
log_dict['u'] = np.stack([log_dict['u1'], log_dict['u2'], log_dict['u3'], log_dict['u4']], axis=1)

# save log dict to flight_data/simulation_logs
# make folder
name = 'INDI_NET_sim_with_gate_penalty'
folder = 'flight_data/simulation_logs'
if not os.path.exists(folder):
    os.makedirs(folder)
# save dict as npz
np.savez(folder + '/' + name, **log_dict)

# Generate C code

In [ ]:
import torch.nn as nn
# first test only moment randomization
# path = 'models/PPO_INDI/INDI_TEST2_no_rate_penalty/375000000'
# path = 'models/PPO_INDI/INDI_TEST2_with_rate_penalty/375000000'
model = PPO.load(path)

# get network
network = list(model.policy.mlp_extractor.policy_net) + [model.policy.action_net]
network = nn.Sequential(*network)
print('NETWORK:')
print(network)

print(model.policy.action_dist)
print(model.policy.log_std)
print(model.policy.log_std.exp())
network_std = model.policy.log_std.exp().cpu().detach().numpy()
print(network_std)


In [ ]:
test_env = Quadcopter3DGates(num_envs=1, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1, pause_if_collision=True)

animate_policy(model, test_env, deterministic=False)

In [ ]:
import torch
import torch.nn as nn
import os
import subprocess

# remove the c_code folder and all of its contents
subprocess.call('rm -rf c_code', shell=True)
# create a new c_code folder
subprocess.call('mkdir c_code', shell=True)

# Create the "c_code" folder if it doesn't exist
output_folder = "c_code"
os.makedirs(output_folder, exist_ok=True)

# Generate the C file and the header file inside the "c_code" folder
source_file_path = os.path.join(output_folder, "neural_network.c")
header_file_path = os.path.join(output_folder, "neural_network.h")

# np.float32 to str
float_to_str = lambda x: str(float(x))

# Generate the C file
with open(source_file_path, "w") as file:
    file.write('#include "neural_network.h"\n')
    file.write("#include <stdio.h>\n")
    file.write("#include <math.h>\n\n")

    # Define weights and biases as global constant float arrays
    i = 1
    for layer in network:
        if isinstance(layer, nn.Linear):
            weights_layer = layer.weight.data.cpu().numpy()
            biases_layer = layer.bias.data.cpu().numpy()

            file.write(f"const float weights_fc{i}[] = {{\n")
            file.write(",\n".join([", ".join(map(float_to_str, row)) for row in weights_layer]))
            file.write("\n};\n\n")

            file.write(f"const float biases_fc{i}[] = {{\n")
            file.write(", ".join(map(float_to_str, biases_layer)))
            file.write("\n};\n\n")

            i+=1

    # LINEAR LAYER
    file.write("void nn_linear(const float* weights, const float* biases, const float* input, int in_features, int out_features, float* output) {\n")
    file.write("    for (int i = 0; i < out_features; ++i) {\n")
    file.write("        float neuron = biases[i];\n")
    file.write("        for (int j = 0; j < in_features; ++j) {\n")
    file.write("            neuron += input[j] * weights[i * in_features + j];\n")
    file.write("        }\n")
    file.write("        output[i] = neuron;\n")
    file.write("    }\n")
    file.write("}\n\n")

    # RELU LAYER
    file.write("void nn_relu(float* input, int size) {\n")
    file.write("    for (int i = 0; i < size; ++i) {\n")
    file.write("        input[i] = fmaxf(0, input[i]);\n")
    file.write("    }\n")
    file.write("}\n\n")

    # TANH LAYER
    file.write("void nn_tanh(float* input, int size) {\n")
    file.write("    for (int i = 0; i < size; ++i) {\n")
    file.write("        input[i] = tanh(input[i]);\n")
    file.write("    }\n")
    file.write("}\n\n")

    # FORWARD FUNCTION
    file.write("void nn_forward(const float* input, float* output) {\n")
    layer_size = network[0].out_features
    num_layers = sum(isinstance(layer, nn.Linear) for layer in network)
    i=0
    input_array = "input"
    for layer in network:
        if isinstance(layer, nn.Linear):
            i+=1
            if i<num_layers:
                file.write(f"    float fc{i}_output[{layer.out_features}];\n")
                file.write(f"    nn_linear(weights_fc{i}, biases_fc{i}, {input_array}, {layer.in_features}, {layer.out_features}, fc{i}_output);\n")
                input_array = f"fc{i}_output"
            else:
                file.write(f"    nn_linear(weights_fc{i}, biases_fc{i}, {input_array}, {layer.in_features}, {layer.out_features}, output);\n")
                input_array = "output"
            layer_size = layer.out_features
        elif isinstance(layer, nn.ReLU):
            file.write(f"    nn_relu({input_array}, {layer_size});\n")
        elif isinstance(layer, nn.Tanh):
            file.write(f"    nn_tanh({input_array}, {layer_size});\n")
        else:
            raise Exception(f"Unsupported layer: {layer}")
    file.write("}\n")

# Generate the header file
with open(header_file_path, "w") as header_file:
    header_file.write("#ifndef NEURAL_NETWORK_H\n")
    header_file.write("#define NEURAL_NETWORK_H\n\n")
    # Declare the forward function in the header file
    header_file.write("void nn_forward(const float* input, float* output);\n")
    header_file.write("\n#endif // NEURAL_NETWORK_H\n")


# Print the generated files
# Print the generated files
print(f"Generated {source_file_path}")
print(f"Generated {header_file_path}")

In [ ]:
name = 'nn_controller'
# Create the "c_code" folder if it doesn't exist
output_folder = "c_code"
os.makedirs(output_folder, exist_ok=True)

# Generate the C file and the header file inside the "c_code" folder
source_file_path = os.path.join(output_folder, f"{name}.c")
header_file_path = os.path.join(output_folder, f"{name}.h")

num_gates = test_env.num_gates
gates_ahead = test_env.gates_ahead

# Generate the header file
with open(header_file_path, "w") as file:
    file.write(f"#ifndef {name.upper()}_H\n")
    file.write(f"#define {name.upper()}_H\n")
    file.write("#define NN_INDI_CMDS\n")
    file.write("\n")
    file.write("#include <stdint.h>\n")
    file.write("#include <stdbool.h>\n")
    file.write("\n")
    file.write(f'#define GATES_AHEAD {gates_ahead}\n')
    file.write(f'#define NUM_GATES {num_gates}\n')
    file.write("\n")
    # include neural network code
    file.write("// Include the neural network code\n")
    file.write("#include \"neural_network.h\"\n")
    file.write("\n")
    file.write("const float gate_pos[NUM_GATES][3];\n")
    file.write("const float gate_yaw[NUM_GATES];\n")
    file.write("const float start_pos[3];\n")
    # file.write("const float gate_pos_rel[NUM_GATES][3];\n")
    # file.write("const float gate_yaw_rel[NUM_GATES];\n")
    file.write("uint8_t target_gate_index;\n")
    file.write("\n")
    # nn_reset function that resets the target gate index
    file.write("void nn_reset(void);\n")
    # nn_control function that that takes as input a float array of size 16 (world_state) and outputs an array of size 4 (rpms)
    file.write("void nn_control(const float world_state[13], float indi_cmd[4]);\n")
    file.write("\n")
    file.write("#endif\n")

# Generate the C file
with open(source_file_path, "w") as file:
    file.write(f"#include \"{name}.h\"\n")
    file.write("#include <math.h>\n")
    file.write("#include <stdlib.h>\n")
    file.write("\n")
    # define boolean to set controller to determistic
    file.write("bool deterministic = false;\n")
    file.write("\n")
    file.write("const float output_std[4] = {\n")
    for i in range(4):
        file.write(f"    {network_std[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the gate positions and headings as const float arrays
    file.write("const float gate_pos[NUM_GATES][3] = {\n")
    for i in range(num_gates):
        file.write(f"    {{{test_env.gate_pos[i][0]}, {test_env.gate_pos[i][1]}, {test_env.gate_pos[i][2]}}},\n")
    file.write("};\n")
    file.write("\n")
    file.write("const float gate_yaw[NUM_GATES] = {\n")
    for i in range(num_gates):
        file.write(f"    {test_env.gate_yaw[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the start pos as a const float array
    file.write("const float start_pos[3] = {\n")
    file.write(f"    {test_env.start_pos[0]}, {test_env.start_pos[1]}, {test_env.start_pos[2]}\n")
    file.write("};\n")
    file.write("\n")
    # define the relative gate positions and headings as const float arrays
    file.write("const float gate_pos_rel[NUM_GATES][3] = {\n")
    for i in range(num_gates):
        file.write(f"    {{{test_env.gate_pos_rel[i][0]}, {test_env.gate_pos_rel[i][1]}, {test_env.gate_pos_rel[i][2]}}},\n")
    file.write("};\n")
    file.write("\n")
    file.write("const float gate_yaw_rel[NUM_GATES] = {\n")
    for i in range(num_gates):
        file.write(f"    {test_env.gate_yaw_rel[i]},\n")
    file.write("};\n")
    file.write("\n")
    # define the target gate index and set it to 0
    file.write("uint8_t target_gate_index = 0;\n")
    file.write("\n")
    file.write("void nn_reset() {\n")
    file.write("    target_gate_index = 0;\n")
    file.write("}\n")
    file.write("\n")
    
    file.write("void nn_control(const float world_state[13], float indi_cmd[4]) {\n")
    file.write("    // Get the current position, velocity and heading\n")
    file.write("    float pos[3] = {world_state[0], world_state[1], world_state[2]};\n")
    file.write("    float vel[3] = {world_state[3], world_state[4], world_state[5]};\n")
    file.write("    float yaw = world_state[8];\n")
    file.write("\n")
    file.write("    // Get the position and heading of the target gate\n")
    file.write("    float target_pos[3] = {gate_pos[target_gate_index][0], gate_pos[target_gate_index][1], gate_pos[target_gate_index][2]};\n")
    file.write("    float target_yaw = gate_yaw[target_gate_index];\n")
    file.write("\n")
    file.write("    // Set the target gate index to the next gate if we passed through the current one\n")
    file.write("    if (cosf(target_yaw) * (pos[0] - target_pos[0]) + sinf(target_yaw) * (pos[1] - target_pos[1]) > 0) {\n")
    file.write("        target_gate_index++;\n")
    file.write("        // loop back to the first gate if we reach the end\n")
    file.write("        target_gate_index = target_gate_index % NUM_GATES;\n")
    file.write("        // reset the target position and heading\n")
    file.write("        target_pos[0] = gate_pos[target_gate_index][0];\n")
    file.write("        target_pos[1] = gate_pos[target_gate_index][1];\n")
    file.write("        target_pos[2] = gate_pos[target_gate_index][2];\n")
    file.write("        target_yaw = gate_yaw[target_gate_index];\n")
    file.write("    }\n")
    file.write("\n")
    file.write("    // Get the position of the drone in gate frame\n")
    file.write("    float pos_rel[3] = {\n")
    file.write("        cosf(target_yaw) * (pos[0] - target_pos[0]) + sinf(target_yaw) * (pos[1] - target_pos[1]),\n")
    file.write("        -sinf(target_yaw) * (pos[0] - target_pos[0]) + cosf(target_yaw) * (pos[1] - target_pos[1]),\n")
    file.write("        pos[2] - target_pos[2]\n")
    file.write("    };\n")
    file.write("\n")
    file.write("    // Get the velocity of the drone in gate frame\n")
    file.write("    float vel_rel[3] = {\n")
    file.write("        cosf(target_yaw) * vel[0] + sinf(target_yaw) * vel[1],\n")
    file.write("        -sinf(target_yaw) * vel[0] + cosf(target_yaw) * vel[1],\n")
    file.write("        vel[2]\n")
    file.write("    };\n")
    file.write("\n")
    file.write("    // Get the heading of the drone in gate frame\n")
    file.write("    float yaw_rel = yaw - target_yaw;\n")
    file.write("    while (yaw_rel > M_PI) {yaw_rel -= 2*M_PI;}\n")
    file.write("    while (yaw_rel < -M_PI) {yaw_rel += 2*M_PI;}\n")
    file.write("\n")
    file.write("    // Get the neural network input\n")
    file.write("    float nn_input[13+4*GATES_AHEAD];\n")
    file.write("    // position and velocity\n")
    file.write("    for (int i = 0; i < 3; i++) {\n")
    file.write("        nn_input[i] = pos_rel[i];\n")
    file.write("        nn_input[i+3] = vel_rel[i];\n")
    file.write("    }\n")
    file.write("    // attitude\n")
    file.write("    nn_input[6] = world_state[6];\n")
    file.write("    nn_input[7] = world_state[7];\n")
    file.write("    nn_input[8] = yaw_rel;\n")
    file.write("    // body rates\n")
    file.write("    nn_input[9] = world_state[9];\n")
    file.write("    nn_input[10] = world_state[10];\n")
    file.write("    nn_input[11] = world_state[11];\n")
    file.write("    // normalized thrust \n")
    file.write(f'    float T_min = {T_min};\n')
    file.write(f'    float T_max = {T_max};\n')
    file.write("    nn_input[12] = (world_state[12] - T_min) / (T_max - T_min) * 2 - 1;\n")
    file.write("\n")
    file.write("    // relative gate positions and headings\n")
    file.write("    for (int i = 0; i < GATES_AHEAD; i++) {\n")
    file.write("        uint8_t index = target_gate_index + i + 1;\n")
    file.write("        // loop back to the first gate if we reach the end\n")
    file.write("        index = index % NUM_GATES;\n")
    file.write("        nn_input[13+4*i]   = gate_pos_rel[index][0];\n")
    file.write("        nn_input[13+4*i+1] = gate_pos_rel[index][1];\n")
    file.write("        nn_input[13+4*i+2] = gate_pos_rel[index][2];\n")
    file.write("        nn_input[13+4*i+3] = gate_yaw_rel[index];\n")
    file.write("    }\n")
    file.write("    // Get the neural network output and write to the action array\n")
    file.write("    float nn_output[4];\n")
    file.write("    nn_forward(nn_input, nn_output);\n")
    file.write("\n")
    # if determinstic is false, add gaussian noise to the output
    file.write("    // add gaussian noise to the output\n")
    file.write("    if (!deterministic) {\n")
    file.write("        for (int i = 0; i < 4; i++) {\n")
    # generate random gaussian variables using the Box–Muller transform
    file.write("            // generate random gaussian variables using the Box–Muller transform\n")
    file.write("            float u1 = (float)rand() / RAND_MAX;\n")
    file.write("            float u2 = (float)rand() / RAND_MAX;\n")
    file.write("            float rand_std = sqrtf(-2 * logf(u1)) * cosf(2 * M_PI * u2);\n")
    file.write("            // add the noise to the output\n")
    file.write("            nn_output[i] += output_std[i] * rand_std;\n")
    file.write("        }\n")
    file.write("    }\n")
    file.write("\n")
    file.write("    for (int i = 0; i < 4; i++) {\n")
    file.write("        // clip the output to the range [-1, 1]\n")
    file.write("        if (nn_output[i] > 1) {nn_output[i] = 1;}\n")
    file.write("        if (nn_output[i] < -1) {nn_output[i] = -1;}\n")
    file.write("    }\n")
    file.write("    // map the output to the correct ranges\n")
    file.write(f"    float p_min = {p_min};\n")
    file.write(f"    float p_max = {p_max};\n")
    file.write(f"    float q_min = {q_min};\n")
    file.write(f"    float q_max = {q_max};\n")
    file.write(f"    float r_min = {r_min};\n")
    file.write(f"    float r_max = {r_max};\n")
    file.write("    indi_cmd[0] = (nn_output[0] + 1) / 2 * (p_max - p_min) + p_min;\n")
    file.write("    indi_cmd[1] = (nn_output[1] + 1) / 2 * (q_max - q_min) + q_min;\n")
    file.write("    indi_cmd[2] = (nn_output[2] + 1) / 2 * (r_max - r_min) + r_min;\n")
    file.write("    indi_cmd[3] = (nn_output[3] + 1) / 2 * (T_max - T_min) + T_min;\n")
    file.write("}\n")

# Print the generated files
print(f"Generated {source_file_path}")
print(f"Generated {header_file_path}")

# Test C code

In [ ]:
import os
import subprocess
import ctypes
import numpy as np
import importlib
importlib.reload(ctypes)

# https://cu7ious.medium.com/how-to-use-dynamic-libraries-in-c-46a0f9b98270
path = os.path.abspath('c_code')
# Create object files
subprocess.call('gcc -fPIC -c *.c', shell=True, cwd=path)
# Create library
subprocess.call('gcc -shared -Wl,-soname,libtools.so -o libtools.so *.o', shell=True, cwd=path)
# Remove object files
subprocess.call('rm *.o', shell=True, cwd=path)

lib_path = os.path.abspath("c_code/libtools.so")
fun = ctypes.CDLL(lib_path)

print(fun.nn_forward)
print(fun.nn_control)

# define argument types 
fun.nn_forward.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float)]
fun.nn_control.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float)]

In [ ]:
def nn_control(x):
    x = np.array(x, dtype=np.float32)

    # scale to Thrust to T_min and T_max
    x[12] = (x[12] + 1)/2*(T_max - T_min) + T_min

    c_net_input = (ctypes.c_float*len(x))(*x)
    c_net_output = (ctypes.c_float*4)()
    
    fun.nn_control(c_net_input, c_net_output)
    out = np.array(c_net_output[:])
    
    # map back to [-1,1]
    out[0] = (out[0] - p_min)/(p_max - p_min)*2 - 1
    out[1] = (out[1] - q_min)/(q_max - q_min)*2 - 1
    out[2] = (out[2] - r_min)/(r_max - r_min)*2 - 1
    out[3] = (out[3] - T_min)/(T_max - T_min)*2 - 1
    return out


In [ ]:
# Simulate C Network
test_env = Quadcopter3DGates(num_envs=1, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, gates_ahead=1, pause_if_collision=True)
test_env.reset()
fun.nn_reset()

action_list = []

def run():
    world_state = test_env.world_states[0]
    action = nn_control(world_state.copy())
    action_list.append(action)
    
    actions = np.array([action])

    states, rewards, dones, infos = test_env.step(actions)

    if dones[0]:
        fun.nn_reset()
    return test_env.render()

animation.view(run, gate_pos=test_env.gate_pos, gate_yaw=test_env.gate_yaw)

In [ ]:
import matplotlib.pyplot as plt

action_list = np.array(action_list)
plt.plot(action_list[:,0], label='p_cmd')
plt.plot(action_list[:,1], label='q_cmd')
plt.plot(action_list[:,2], label='r_cmd')
plt.legend()
plt.show()

plt.plot(action_list[:,3], label='T_cmd')
plt.legend()
plt.show()